In [ ]:
!pip install transformers datasets torch scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 72.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 109.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [ ]:
import pandas as pd
from transformers import BertTokenizer, AutoTokenizer
from sklearn.model_selection import train_test_split

from google.colab import files
uploaded = files.upload()

# Load and preprocess the dataset
df = pd.read_csv("final_dataset_make_call_unique_clean.csv")  # Your dataset file

# Convert BIO labels and intent labels to numeric
intent_labels = {'money_transfer': 0, 'out_of_scope': 1, 'make_call': 2, 'affirmative': 3, 'negative': 4, 'greeting': 5, 'more_options': 6, 'angry': 7}

entity_labels_dict = {
    'O': 0,
    'B-amount': 1,
    'I-amount': 2,
    'B-contact_name': 3,
    'I-contact_name': 4,
    'B-payment_platform': 5,
    'I-payment_platform': 6
}


def process_row(row):
    tokens = row[2::2]  # Tokens are in alternate columns
    labels = row[3::2]  # BIO labels
    # Convert tokens to strings and handle NaN values
    tokens = [str(token) if pd.notna(token) else "" for token in tokens]
    # Convert labels to numeric values using the global entity_labels dictionary
    numeric_labels = [entity_labels_dict.get(label, 0) if pd.notna(label) else 0 for label in labels] # Use entity_labels_dict instead of entity_labels

    return tokens, numeric_labels

data = []
for _, row in df.iterrows():
    #print(f"Total data: {data}")
    tokens, entity_labels = process_row(row)
    data.append({
        "text": " ".join(tokens),
        "tokens": tokens,
        "intent": intent_labels[row["Intent"]],
        "entity_labels": entity_labels
    })

from transformers import BertModel
import torch.nn as nn
import torch

class BertForIntentAndEntities(nn.Module):
    def __init__(self, pretrained_model_name, num_intent_labels, num_entity_labels):
        super().__init__()
        self.bert = BertModel.from_pretrained(pretrained_model_name)
        self.dropout = nn.Dropout(0.1)  # <-- Added dropout
        self.intent_classifier = nn.Linear(self.bert.config.hidden_size, num_intent_labels)
        self.entity_classifier = nn.Linear(self.bert.config.hidden_size, num_entity_labels)

    def forward(self, input_ids, attention_mask, token_type_ids=None, labels=None, intent_labels=None):
        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )
        sequence_output = self.dropout(outputs.last_hidden_state)   # <-- Added dropout here
        pooled_output = self.dropout(outputs.pooler_output)

        intent_logits = self.intent_classifier(pooled_output)
        entity_logits = self.entity_classifier(sequence_output)

        loss = None
        if labels is not None and intent_labels is not None:
            intent_loss_fn = nn.CrossEntropyLoss()
            entity_loss_fn = nn.CrossEntropyLoss(ignore_index=-100)  # <-- ignores padding tokens

            intent_loss = intent_loss_fn(intent_logits, intent_labels)
            entity_loss = entity_loss_fn(entity_logits.view(-1, entity_logits.shape[-1]), labels.view(-1))

            # <-- CHANGE: give more weight to entity loss
            loss = 0.3 * intent_loss + 0.7 * entity_loss

        return {"loss": loss, "intent_logits": intent_logits, "entity_logits": entity_logits}


# Split into train/test sets
train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased", use_fast=True) # Load the fast tokenizer using AutoTokenizer

def tokenize_and_align_labels(example):
    # Tokenize the input tokens
    tokenized_inputs = tokenizer(
        example["tokens"],
        truncation=True,
        padding="max_length",
        max_length=20,
        is_split_into_words=True,
        return_tensors="pt"
    )
    # Align labels with the tokenized word IDs
    word_ids = tokenized_inputs.word_ids()
    # labels = [-100 if word_id is None else example["entity_labels"][word_id] for word_id in word_ids]
    previous_word_id = None
    labels = []

    for word_id in word_ids:
        if word_id is None:
            labels.append(-100)
        elif word_id != previous_word_id:
            labels.append(example["entity_labels"][word_id])  # First subword gets label
        else:
            labels.append(-100)  # Subword — ignore in loss
        previous_word_id = word_id
    tokenized_inputs["labels"] = torch.tensor(labels)
    tokenized_inputs["intent_labels"] = torch.tensor(example["intent"])

    # Print original text and tokenized inputs
    original_text = " ".join(example["tokens"])  # Combine tokens into a single text string
    # print(f"Original Text: {original_text}")
    # print(f"Tokenized inputs: {tokenized_inputs}")

    return tokenized_inputs

train_dataset = [tokenize_and_align_labels(example) for example in train_data]
val_dataset = [tokenize_and_align_labels(example) for example in val_data]

Saving final_dataset_make_call_unique_clean.csv to final_dataset_make_call_unique_clean (1).csv


In [ ]:
from torch.utils.data import Dataset, DataLoader # Import Dataset and DataLoader

# Define the dataset class
class TokenizedDataset(Dataset):
    def __init__(self, tokenized_data):
        self.tokenized_data = tokenized_data

    def __len__(self):
        return len(self.tokenized_data["input_ids"])

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.tokenized_data.items()}

# Convert tokenized data to dictionary format
train_data = {
    "input_ids": [example["input_ids"].squeeze() for example in train_dataset],
    "attention_mask": [example["attention_mask"].squeeze() for example in train_dataset],
    "labels": [example["labels"].squeeze() for example in train_dataset],  # Wrap labels list in torch.tensor()
    "intent_labels": [example["intent_labels"].squeeze() for example in train_dataset]
}
val_data = {
    "input_ids": [example["input_ids"].squeeze() for example in val_dataset],
    "attention_mask": [example["attention_mask"].squeeze() for example in val_dataset],
    "labels": [example["labels"].squeeze() for example in val_dataset],  # Wrap labels list in torch.tensor()
    "intent_labels": [example["intent_labels"].squeeze() for example in val_dataset]
}

# Wrap into PyTorch Dataset
train_dataset = TokenizedDataset(train_data)
val_dataset = TokenizedDataset(val_data)

# Create DataLoaders
batch_size = 8
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

/tmp/ipython-input-25-2365521852.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.tokenized_data.items()}


In [ ]:
import torch
from torch.optim import AdamW # Import AdamW from torch.optim
from torch.utils.data import DataLoader
from tqdm import tqdm

from transformers import BertForTokenClassification

# Load model
num_intent_labels = len(intent_labels)
num_entity_labels = len(entity_labels_dict)
model = BertForIntentAndEntities("bert-base-uncased", num_intent_labels, num_entity_labels)


# Move model to the appropriate device (GPU or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

BertForIntentAndEntities(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, 

In [ ]:
from torch.optim import AdamW

# Define the optimizer
optimizer = AdamW(model.parameters(), lr=2e-5)

In [ ]:
from torch.nn.utils import clip_grad_norm_

# Training loop
epochs = 10
for epoch in range(epochs):
    model.train()
    total_loss = 0

    for batch in tqdm(train_loader):
        # Move batch to device
        batch = {key: val.to(device) for key, val in batch.items()}

        # Forward pass
        outputs = model(
            input_ids=batch["input_ids"],
            attention_mask=batch["attention_mask"],
            labels=batch["labels"],
            intent_labels=batch["intent_labels"]
        )

        # Compute loss
        loss = outputs["loss"]
        total_loss += loss.item()

        # Backward pass
        loss.backward()

        # Gradient clipping
        clip_grad_norm_(model.parameters(), max_norm=1.0)

        # Optimizer step
        optimizer.step()
        optimizer.zero_grad()

    # Print epoch loss
    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1}/{epochs}, Loss: {avg_loss:.4f}")


  0%|          | 0/9 [00:00<?, ?it/s]/tmp/ipython-input-30-2365521852.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.tokenized_data.items()}
100%|██████████| 9/9 [00:00<00:00, 11.63it/s]


Epoch 1/10, Loss: 1.6869


100%|██████████| 9/9 [00:00<00:00, 13.82it/s]


Epoch 2/10, Loss: 1.1527


100%|██████████| 9/9 [00:00<00:00, 13.71it/s]


...

Epoch 8/10, Loss: 0.2900


100%|██████████| 9/9 [00:00<00:00, 13.69it/s]


Epoch 9/10, Loss: 0.2386


100%|██████████| 9/9 [00:00<00:00, 13.63it/s]

Epoch 10/10, Loss: 0.2129


In [ ]:
from sklearn.metrics import classification_report, accuracy_score

# Evaluation loop
model.eval()  # Set the model to evaluation mode
all_entity_preds, all_entity_labels = [], []
all_intent_preds = []  # Initialize as empty list
all_intent_labels = []  # Initialize as empty list

with torch.no_grad():  # Disable gradient computation
    for batch in val_loader:
        # Move batch to device
        batch = {key: val.to(device) for key, val in batch.items()}

        # Forward pass
        outputs = model(
            input_ids=batch["input_ids"],
            attention_mask=batch["attention_mask"]
        )

        # Extract intent logits and predictions
        intent_logits = outputs["intent_logits"]
        intent_predictions = torch.argmax(intent_logits, dim=1)  # Intent predictions
        all_intent_preds.extend(intent_predictions.cpu().numpy())
        all_intent_labels.extend(batch["intent_labels"].cpu().numpy())

        # Extract entity logits and predictions
        entity_logits = outputs["entity_logits"]
        entity_predictions = torch.argmax(entity_logits, dim=2)  # Entity predictions

        # Store entity predictions and labels
        all_entity_preds.extend(entity_predictions.cpu().numpy())
        all_entity_labels.extend(batch["labels"].cpu().numpy())


# Flatten and filter out padding tokens (-100) for entities
entity_preds_flat = []
entity_labels_flat = []

for pred, label in zip(all_entity_preds, all_entity_labels):
    for p, l in zip(pred, label):
        if l != -100:  # Ignore padding tokens
            entity_preds_flat.append(p)
            entity_labels_flat.append(l)

# Identify the unique intent labels in the validation data
unique_labels = sorted(set(all_intent_labels))  # Unique labels in true data

# Filter target names for available labels
available_target_names = [key for key, val in intent_labels.items() if val in unique_labels]

#Generate intent classification report with only available labels
print("Intent Classification Report:")
print(
    classification_report(
        all_intent_labels,
        all_intent_preds,
        target_names=available_target_names,
        labels=unique_labels
    )
)

# Generate entity extraction report
print("\nEntity Extraction Report:")
print(classification_report(entity_labels_flat, entity_preds_flat))



Intent Classification Report:
                precision    recall  f1-score   support

money_transfer       1.00      1.00      1.00         4
  out_of_scope       1.00      1.00      1.00         1
     make_call       1.00      1.00      1.00         7
   affirmative       1.00      1.00      1.00         1
      negative       1.00      1.00      1.00         1
      greeting       0.50      1.00      0.67         1
         angry       1.00      0.50      0.67         2

      accuracy                           0.94        17
     macro avg       0.93      0.93      0.90        17
  weighted avg       0.97      0.94      0.94        17


Entity Extraction Report:
              precision    recall  f1-score   support

           0       0.98      0.94      0.96        53
           1       1.00      0.75      0.86         4
           2       0.80      1.00      0.89         4
           3       0.79      1.00      0.88        11
           4       1.00      1.00      1.00        11

/tmp/ipython-input-30-2365521852.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.tokenized_data.items()}


In [ ]:
import os

# Create the directory if it doesn't exist
os.makedirs("./trained_model", exist_ok=True)

# Save the model's state dictionary
torch.save(model.state_dict(), "./trained_model/pytorch_model.bin")

entity_labels_dict = {
    'O': 0,
    'B-amount': 1,
    'I-amount': 2,
    'B-contact_name': 3,
    'I-contact_name': 4,
    'B-payment_platform': 5,
    'I-payment_platform': 6
}

# Save the model configuration
model_config = {
    "pretrained_model_name": "bert-base-uncased",
    "num_intent_labels": len(intent_labels),
    "num_entity_labels": len(entity_labels_dict)
}
import json
with open("./trained_model/config.json", "w") as f:
    json.dump(model_config, f)

# Save the tokenizer
tokenizer.save_pretrained("./trained_model")


('./trained_model/tokenizer_config.json',
 './trained_model/special_tokens_map.json',
 './trained_model/vocab.txt',
 './trained_model/added_tokens.json',
 './trained_model/tokenizer.json')

In [ ]:
# Original model checking code
import json
from transformers import BertModel, BertTokenizer

# Load the model configuration
with open("./trained_model/config.json", "r") as f:
    model_config = json.load(f)

# Reinitialize the model
model = BertForIntentAndEntities(
    pretrained_model_name=model_config["pretrained_model_name"],
    num_intent_labels=model_config["num_intent_labels"],
    num_entity_labels=model_config["num_entity_labels"]
)

# Load the model's state dictionary
model.load_state_dict(torch.load("./trained_model/pytorch_model.bin"))

# Load the tokenizer
tokenizer = BertTokenizer.from_pretrained("./trained_model")

# Move to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

<ipython-input-10-c7fd62265be2>:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("./trained_model/pytorch_model.bin"))


BertForIntentAndEntities(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, 

In [13]:
# drive model checking code


from transformers import BertModel
import torch.nn as nn
import torch

class BertForIntentAndEntities(nn.Module):
    def __init__(self, pretrained_model_name, num_intent_labels, num_entity_labels):
        super().__init__()
        self.bert = BertModel.from_pretrained(pretrained_model_name)
        self.dropout = nn.Dropout(0.1)  # <-- Added dropout
        self.intent_classifier = nn.Linear(self.bert.config.hidden_size, num_intent_labels)
        self.entity_classifier = nn.Linear(self.bert.config.hidden_size, num_entity_labels)

    def forward(self, input_ids, attention_mask, token_type_ids=None, labels=None, intent_labels=None):
        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )
        sequence_output = self.dropout(outputs.last_hidden_state)   # <-- Added dropout here
        pooled_output = self.dropout(outputs.pooler_output)

        intent_logits = self.intent_classifier(pooled_output)
        entity_logits = self.entity_classifier(sequence_output)

        loss = None
        if labels is not None and intent_labels is not None:
            intent_loss_fn = nn.CrossEntropyLoss()
            entity_loss_fn = nn.CrossEntropyLoss(ignore_index=-100)  # <-- ignores padding tokens

            intent_loss = intent_loss_fn(intent_logits, intent_labels)
            entity_loss = entity_loss_fn(entity_logits.view(-1, entity_logits.shape[-1]), labels.view(-1))

            # <-- CHANGE: give more weight to entity loss
            loss = 0.3 * intent_loss + 0.7 * entity_loss

        return {"loss": loss, "intent_logits": intent_logits, "entity_logits": entity_logits}

import os

# Path to your saved model in Drive
model_dir = "/content/drive/MyDrive/model_folder_unique"

import json
from transformers import BertModel, BertTokenizer

# Load the model configuration
with open(os.path.join(model_dir, "config.json"), "r") as f:
    model_config = json.load(f)

# Reinitialize the model
model = BertForIntentAndEntities(
    pretrained_model_name=model_config["pretrained_model_name"],
    num_intent_labels=model_config["num_intent_labels"],
    num_entity_labels=model_config["num_entity_labels"]
)

# Load the model's state dictionary
model.load_state_dict(torch.load(os.path.join(model_dir, "pytorch_model.bin"), map_location=torch.device('cpu')))

# Load the tokenizer
tokenizer = BertTokenizer.from_pretrained(model_dir)

# Move to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

BertForIntentAndEntities(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, 

In [17]:
def predict_intent_and_entities(text):
    import re
    # Input text

    # Tokenize the input
    tokens = tokenizer(
        text.split(),
        truncation=True,
        padding="max_length",
        max_length=20,
        is_split_into_words=True,
        return_tensors="pt"
    )
    #print(tokens)

    tokens = {key: val.to(device) for key, val in tokens.items()}

    #print(tokens)

    # Perform inference
    with torch.no_grad():
        outputs = model(**tokens)

    # Get logits for intent and entities
    entity_logits = outputs["entity_logits"]  # Entity logits
    intent_logits = outputs["intent_logits"]  # Intent logits

    # Intent prediction
    intent_prediction = torch.argmax(intent_logits, dim=1)
    intent_labels = {
        0: 'money_transfer',
        1: 'out_of_scope',
        2: 'make_call',
        3: 'affirmative',
        4: 'negative',
        5: 'greeting',
        6: 'more_options',
        7: 'angry'
    }
    predicted_intent = intent_labels[intent_prediction.item()]
    print(f"Predicted Intent: {predicted_intent}")

    # Entity prediction
    entity_predictions = torch.argmax(entity_logits, dim=2)
    #print(entity_predictions)


    # Mapping from label ID to label name
    id2label = {
        0: 'O',
        1: 'B-amount',
        2: 'I-amount',
        3: 'B-contact_name',
        4: 'I-contact_name',
        5: 'B-payment_platform',
        6: 'I-payment_platform'
    }

    # Convert predictions to labels
    predicted_labels = [id2label[label_id] for label_id in entity_predictions[0].cpu().numpy()]
    #print(predicted_labels)

    # Convert token IDs to tokens
    tokens_text = tokenizer.convert_ids_to_tokens(tokens["input_ids"][0])
    #print(tokens_text)

    # Combine tokens with their predicted labels
    results = [(token, label) for token, label in zip(tokens_text, predicted_labels) if token not in ["[CLS]", "[SEP]", "[PAD]"]]

    # Print results
    for token, label in results:
        print(f"{token:15} -> {label}")

    entities = {}
    current_entity = None
    entity_tokens = []

    for token, label in results:
        print(f"Token: {token}, Label: {label}")
        if label.startswith("B-"):  # Beginning of a new entity
            if current_entity:
                if current_entity == label[2:]:
                    if token.startswith("##"):
                        # Subword continuation: merge into previous token
                        entity_tokens[-1] += token[2:]  # Append subword without "##"
                    else:
                        entity_tokens.append(token)  # Add new token
                else:
                    # Add the completed entity to the result
                    entities[current_entity] = " ".join(entity_tokens)
                    current_entity = None
                    entity_tokens = []
                    entity_type = label[2:]  # Extract entity type (e.g., 'amount')
                    current_entity = entity_type
                    entity_tokens = [token]  # Start a new entity
            else:
              entity_type = label[2:]  # Extract entity type (e.g., 'amount')
              current_entity = entity_type
              entity_tokens = [token]  # Start a new entity
        elif label.startswith("I-") and current_entity == label[2:]:  # Continuation of the same entity
            if token.startswith("##"):
                # Subword continuation: merge into previous token
                entity_tokens[-1] += token[2:]  # Append subword without "##"
            else:
                entity_tokens.append(token)  # Add new token
        else:
            if current_entity:
                # Add the completed entity to the result
                entities[current_entity] = " ".join(entity_tokens)
            current_entity = None
            entity_tokens = []

        # Print the variables at each iteration
        # print(f"Current Entity: {current_entity}")
        # print(f"Entity Tokens: {entity_tokens}")
        # print(f"Entities: {entities}")
        # print("-" * 40)  # Separator for readability

    # Add the last entity (if any)
    if current_entity:
        entities[current_entity] = " ".join(entity_tokens)

    print("Final Extracted Entities:", entities)

    # Check if 'amount' key exists in the entities
    if 'amount' in entities:
        numbers = re.findall(r'\d+', entities['amount'])
        if "one" in entities['amount']:
            if numbers:
                total_amount = sum(map(int, numbers)) + 1
            else:
                total_amount = 1
        else:
            total_amount = sum(map(int, numbers)) if numbers else 0
        entities['amount'] = str(total_amount)

    print("Updated Entities:", entities)



In [15]:
# prompt: delete trained_model folder with all file within

import shutil
import os

# Check if the directory exists before attempting to remove it
if os.path.exists("./trained_model"):
  shutil.rmtree("./trained_model")
  print("Directory 'trained_model' and its contents removed successfully.")
else:
  print("Directory 'trained_model' does not exist.")

Directory 'trained_model' does not exist.


In [18]:
from IPython.display import display, clear_output
import ipywidgets as widgets

# Create input box
text_box = widgets.Text(
    value='',
    placeholder='Type your input and press Enter',
    description='Input:',
    disabled=False
)

# Define the function to call on Enter key press
def on_enter_pressed(sender):
    clear_output(wait=True)  # Clear previous output
    display(text_box)        # Redisplay the input box
    text = sender.value
    predict_intent_and_entities(text)  # Your prediction function

# Bind the event
text_box.on_submit(on_enter_pressed)

# Display the input box initially
display(text_box)


Text(value='send money to daru ki dukan', description='Input:', placeholder='Type your input and press Enter')

Predicted Intent: money_transfer
send            -> O
money           -> O
to              -> O
dar             -> B-contact_name
##u             -> I-contact_name
ki              -> I-contact_name
du              -> B-contact_name
##kan           -> I-contact_name
Token: send, Label: O
Token: money, Label: O
Token: to, Label: O
Token: dar, Label: B-contact_name
Token: ##u, Label: I-contact_name
Token: ki, Label: I-contact_name
Token: du, Label: B-contact_name
Token: ##kan, Label: I-contact_name
Final Extracted Entities: {'contact_name': 'daru ki dukan'}
Updated Entities: {'contact_name': 'daru ki dukan'}


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os
import torch
import shutil

# Path to the source zip file (in Colab workspace)
source_zip = "trained_model"

# Path to save in Google Drive
destination_zip = "/content/drive/MyDrive/model_folder_unique"
destination = '/content/drive/MyDrive/model_folder_unique.zip'

if os.path.exists(destination):
  os.remove(destination)
  print(f"File '{destination}' deleted successfully.")
else:
  print(f"File '{destination}' does not exist.")

if os.path.exists(destination_zip):
  # If it exists, remove it to avoid the FileExistsError
  shutil.rmtree(destination_zip)
  print(f"Existing directory '{destination_zip}' removed.")
else:
  print(f"Directory '{destination_zip}' does not exist.")

# Move the file to Google Drive
shutil.copytree(source_zip, destination_zip)

print(f"File moved to Google Drive at {destination_zip}")

# Replace 'my_folder' with your folder name and 'my_folder.zip' with the desired zip name
shutil.make_archive('/content/drive/MyDrive/model_folder_unique', 'zip', '/content/drive/MyDrive/model_folder_unique')

print(f"File moved to Google Drive at {destination_zip}")
shutil.copy(destination, '/content/')

'/content/drive/MyDrive/model_folder_unique.zip'

In [6]:
!apt-get install openssh-client
from google.colab import files
uploaded = files.upload()
!mkdir -p ~/.ssh # if already created then......
!chmod 700 ~/.ssh
!chmod 400 /content/aws_ec2_key.pem
!ssh-keyscan -H 13.232.75.140 >> ~/.ssh/known_hosts

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
openssh-client is already the newest version (1:8.9p1-3ubuntu0.13).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


Saving aws_ec2_key.pem to aws_ec2_key (1).pem
# 13.232.75.140:22 SSH-2.0-OpenSSH_8.7
# 13.232.75.140:22 SSH-2.0-OpenSSH_8.7
# 13.232.75.140:22 SSH-2.0-OpenSSH_8.7
# 13.232.75.140:22 SSH-2.0-OpenSSH_8.7
# 13.232.75.140:22 SSH-2.0-OpenSSH_8.7


In [8]:
!scp -i aws_ec2_key.pem /content/drive/MyDrive/model_folder_unique.zip ec2-user@13.232.75.140:/home/ec2-user/trained_model/